#import files


In [ ]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
source=pd.ExcelFile('BW_CPFR_OUTLET_HU10.xls')
sales=pd.read_excel('BW_SDS_HU10(006).xls',header=1)
writer = ExcelWriter('Top5NS_Report.xlsx')


In [ ]:
sales.head()

,Calendar Year/Month,Calendar Year/Week,Customer,Europe Prod Hier 3,Material,Material Name,Sales Quantity,P5 Net Sales EUR,P5 Net Sales LC
0,9.2013,36.2013,290010221,SCPCTMELCD,KDL40R470ABAEP,"40"" CONSUMER LCD",1,3.46,1042.16
1,9.2013,36.2013,290010221,SCPIITESLT,SGPT131E2/S.E2,SGPT131E2/S,20,73.46,21752.97
2,9.2013,36.2013,290010320,SCPCVSEEV1,MEXBT4100U.EUR,BT STEPUP 1WIRE,6,20.91,6248.67
3,9.2013,36.2013,290010320,SCPCVSEEV1,XMGTX6041.EUR,GTX SERIES 60WX4CH,2,18.59,5586.49
4,9.2013,36.2013,290010320,SCPCVSEEV1,XSGT1328F.EUR,GT-F SPEAKER 13CM 2WAY,10,4.19,1250.35


In [ ]:
def create_avg_sell(sales):
  ''' Create avg sell price for each material''' 
  unique_material= sales['Material'].unique()
  num_of_unique_products = len(unique_material)
  prod_avg={}
  for item in unique_material:
    small_sales=sales[sales['Material']==item]
    total_sales=small_sales['P5 Net Sales EUR'].sum()
    total_quantity=small_sales['Sales Quantity'].sum()
    if total_quantity>0:
      avg = round(total_sales/total_quantity,4) 
      prod_avg[item]=round(avg,4)
  return prod_avg

In [ ]:
def create_top_products(current_sheet,dict_prod_avg):
  material_list_current_sheet=current_sheet['Material'].unique()
  dict_sellout_NS={}
  for item in material_list_current_sheet:
    filtered_sheet= current_sheet[current_sheet['Material']==item]
    sell_through_quantity = filtered_sheet['Sellthru Qty'].sum()
    dict_sellout_NS[item]=round(sell_through_quantity*dict_prod_avg.get(item,0),2)
  sorted_sellout = sorted(dict_sellout_NS.items(), key=lambda x: (-x[1],x[0]))
  return sorted_sellout
  


In [ ]:
def create_sell_ratio(df_weekly_sale,sorted_sellout):
  total_weekly_sale= df_weekly_sale['P5 Net Sales EUR'].sum()
  product_profit_ratios=[]
  if total_weekly_sale>0:
    for i in range(len(sorted_sellout)):
      product_profit_ratios.append(round(sorted_sellout[i][1]/total_weekly_sale*100,4))
  return product_profit_ratios

In [ ]:
def create_customer_list(df_weekly_sale):
  dict_customer_earn={}
  unique_customers = df_weekly_sale['Customer'].unique()
  for cus in unique_customers:
    df_weekly_sale_for_customers = df_weekly_sale[df_weekly_sale['Customer']==cus]
    dict_customer_earn[cus]=round(df_weekly_sale_for_customers['P5 Net Sales EUR'].sum(),2)
  sorted_customer_earn = sorted(dict_customer_earn.items(), key = lambda x: (-x[1],x[0]))
  return sorted_customer_earn

#main procedure


In [ ]:
dict_prod_avg=create_avg_sell(sales)

In [ ]:
sheets=source.sheet_names

In [ ]:
sheets[0]

'3'

In [ ]:
sheets=source.sheet_names
for sheet in sheets:
  df_current_sheet = source.parse(sheet_name=sheet)
  list_sorted_sellout=create_top_products(df_current_sheet,dict_prod_avg)
  week=df_current_sheet['Calendar Year/Week'][1]
  df_weekly_sale = sales[sales['Calendar Year/Week']==week]
  list_sell_ratio=create_sell_ratio(df_weekly_sale,list_sorted_sellout)
  list_sorted_customer_earn=create_customer_list(df_weekly_sale)
  output=[]
  columns_names=['Week','Top_5_products','Average_sellout','Net_sell_ratios','To5_5_Customers','Customer_Earnings']
  for i in range(5):
    row = [week,list_sorted_sellout[i][0],list_sorted_sellout[i][1],list_sell_ratio[i],list_sorted_customer_earn[i][0],list_sorted_customer_earn[i][1]]
    output.append(row)
  df_top_5_NS= pd.DataFrame(output,columns=columns_names)
  df_top_5_NS.to_excel(writer,sheet_name=str(week),index=False)



In [ ]:
writer.save()